In [1]:
# !pip install numpy==1.26.4

In [1]:
# Uninstall incompatible versions
!pip uninstall --yes torch torchvision torchaudio -q

In [2]:
# Install compatible PyTorch 2.0.1 with CUDA 11.7
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu117 -q

# Install other dependencies
!pip install onnx==1.14.0 onnxruntime==1.15.1 pycocotools pyyaml scipy transformers -q

# CRITICAL: Fix NumPy version
!pip install numpy==1.23.5 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnxruntime 1.15.1 requires numpy>=1.24.2, but you have numpy 1.23.5 which

In [2]:
# 1. Verify GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 1. Clone the official repository into the local Colab space (faster than Drive)
%cd /content/
!git clone https://github.com/lyuwenyu/RT-DETR.git

# 2. Navigate to the PyTorch implementation folder
%cd /content/RT-DETR/rtdetr_pytorch

/content
fatal: destination path 'RT-DETR' already exists and is not an empty directory.
/content/RT-DETR/rtdetr_pytorch


In [4]:
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchdata 0.11.0
Uninstalling torchdata-0.11.0:
  Successfully uninstalled torchdata-0.11.0


In [5]:
# # Install compatible dependencies
# # The RT-DETR repo requires specific versions, but we'll use newer compatible ones
# !pip install -q pycocotools scipy pyyaml opencv-python tqdm

In [5]:
# # 3. Install dependencies
!pip install -r requirements.txt

In [6]:
# Check current PyTorch version
import torch
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Current PyTorch version: 2.0.1+cu117
CUDA available: False


In [9]:
# Extract datasets from Google Drive to local storage (faster)
import shutil
import os

drive_base = '/content/drive/MyDrive/FYP_datasets/Vision-RGB/DUT'
datasets = {'train': 'train.zip', 'val': 'val.zip', 'test': 'test.zip'}

for split, zip_file in datasets.items():
    zip_path = f'{drive_base}/{zip_file}'
    local_zip = f'/content/DUT-{split}.zip'
    local_extract = f'/content/DUT-{split}'

    if os.path.exists(zip_path) and not os.path.exists(local_extract):
        print(f"📦 Extracting {split}...")
        shutil.copy(zip_path, local_zip)
        !unzip -q {local_zip} -d {local_extract}
        os.remove(local_zip)

print("✅ Datasets extracted")

✅ Datasets extracted


In [10]:
# Verify XML structure
import xml.etree.ElementTree as ET

test_xml_dir = '/content/DUT-train/train/xml'
sample_files = os.listdir(test_xml_dir)[:3]

print("Checking sample XML files:\n")
for sample_xml in sample_files:
    tree = ET.parse(os.path.join(test_xml_dir, sample_xml))
    root = tree.getroot()

    print(f"File: {sample_xml}")
    for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        print(f"  Object: {name}, BBox: [{bndbox.find('xmin').text}, {bndbox.find('ymin').text}, {bndbox.find('xmax').text}, {bndbox.find('ymax').text}]")
    print()

Checking sample XML files:

File: 03094.xml
  Object: UAV, BBox: [1178, 698, 1237, 745]

File: 02380.xml
  Object: UAV, BBox: [917, 483, 955, 512]

File: 04691.xml
  Object: UAV, BBox: [857, 598, 873, 603]



In [11]:
# Convert XML to COCO Format
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import cv2

CLASS_MAP = {'UAV': 1}

def convert_xml_to_coco(image_dir, xml_dir, output_file, split_name):
    """Convert Pascal VOC XML annotations to COCO JSON format"""

    images = []
    annotations = []
    categories = [{"id": 1, "name": "uav", "supercategory": "vehicle"}]

    annotation_id = 1
    image_id = 1

    print(f"\n🔄 Converting {split_name} dataset...")
    print(f"  Images: {image_dir}")
    print(f"  XMLs: {xml_dir}")

    if not os.path.exists(image_dir) or not os.path.exists(xml_dir):
        print(f"  ❌ Directory not found!")
        return

    # Get all image files
    valid_images = [f for f in os.listdir(image_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"  Found {len(valid_images)} images")

    skipped = 0

    for filename in tqdm(valid_images, desc=f"Processing {split_name}"):
        xml_filename = os.path.splitext(filename)[0] + '.xml'
        xml_path = os.path.join(xml_dir, xml_filename)
        img_path = os.path.join(image_dir, filename)

        if not os.path.exists(xml_path):
            skipped += 1
            continue

        # Parse XML
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()
        except ET.ParseError:
            skipped += 1
            continue

        # Get image dimensions
        size_node = root.find('size')
        if size_node is not None:
            width = int(size_node.find('width').text)
            height = int(size_node.find('height').text)
        else:
            img = cv2.imread(img_path)
            if img is None:
                skipped += 1
                continue
            height, width = img.shape[:2]

        images.append({
            "id": image_id,
            "file_name": filename,
            "height": height,
            "width": width
        })

        # Extract annotations
        for obj in root.findall('object'):
            name = obj.find('name').text
            class_id = CLASS_MAP.get(name)

            if class_id is None:
                continue

            bndbox = obj.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            bbox_width = xmax - xmin
            bbox_height = ymax - ymin

            if bbox_width <= 0 or bbox_height <= 0:
                continue

            annotations.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": class_id,
                "bbox": [xmin, ymin, bbox_width, bbox_height],
                "area": bbox_width * bbox_height,
                "iscrowd": 0
            })
            annotation_id += 1

        image_id += 1

    # Create COCO format dict
    coco_format = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    # Save JSON
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w') as f:
        json.dump(coco_format, f)

    print(f"  ✓ {len(images)} images, {len(annotations)} annotations")
    if skipped > 0:
        print(f"  ⚠️  Skipped {skipped} files")
    print(f"  Saved to: {output_file}\n")

# Convert all datasets
base_dir = '/content'
for split in ['train', 'val', 'test']:
    dataset_dir = f'{base_dir}/DUT-{split}/{split}'

    if not os.path.exists(dataset_dir):
        print(f"⚠️  Skipping {split} - not found")
        continue

    convert_xml_to_coco(
        image_dir=f'{dataset_dir}/img',
        xml_dir=f'{dataset_dir}/xml',
        output_file=f'{base_dir}/DUT-{split}/annotations/{split}.json',
        split_name=split
    )

print("✅ All datasets converted to COCO format!")


🔄 Converting train dataset...
  Images: /content/DUT-train/train/img
  XMLs: /content/DUT-train/train/xml
  Found 5200 images


Processing train: 100%|██████████| 5200/5200 [00:00<00:00, 12770.35it/s]


  ✓ 5200 images, 5243 annotations
  Saved to: /content/DUT-train/annotations/train.json


🔄 Converting val dataset...
  Images: /content/DUT-val/val/img
  XMLs: /content/DUT-val/val/xml
  Found 2600 images


Processing val: 100%|██████████| 2600/2600 [00:00<00:00, 13672.16it/s]


  ✓ 2600 images, 2620 annotations
  Saved to: /content/DUT-val/annotations/val.json


🔄 Converting test dataset...
  Images: /content/DUT-test/test/img
  XMLs: /content/DUT-test/test/xml
  Found 2200 images


Processing test: 100%|██████████| 2200/2200 [00:00<00:00, 13493.39it/s]


  ✓ 2200 images, 2245 annotations
  Saved to: /content/DUT-test/annotations/test.json

✅ All datasets converted to COCO format!


In [12]:
# Verify COCO format
import json

for split in ['train', 'val', 'test']:
    json_path = f'/content/DUT-{split}/annotations/{split}.json'

    if not os.path.exists(json_path):
        continue

    with open(json_path, 'r') as f:
        data = json.load(f)

    print(f"\n{split.upper()} Dataset:")
    print(f"  Images: {len(data['images'])}")
    print(f"  Annotations: {len(data['annotations'])}")
    print(f"  Categories: {data['categories']}")

    if len(data['annotations']) > 0:
        sample = data['annotations'][0]
        print(f"  Sample annotation: {sample}")


TRAIN Dataset:
  Images: 5200
  Annotations: 5243
  Categories: [{'id': 1, 'name': 'uav', 'supercategory': 'vehicle'}]
  Sample annotation: {'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [1073.0, 664.0, 44.0, 22.0], 'area': 968.0, 'iscrowd': 0}

VAL Dataset:
  Images: 2600
  Annotations: 2620
  Categories: [{'id': 1, 'name': 'uav', 'supercategory': 'vehicle'}]
  Sample annotation: {'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [546.0, 417.0, 38.0, 33.0], 'area': 1254.0, 'iscrowd': 0}

TEST Dataset:
  Images: 2200
  Annotations: 2245
  Categories: [{'id': 1, 'name': 'uav', 'supercategory': 'vehicle'}]
  Sample annotation: {'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [650.0, 385.0, 38.0, 30.0], 'area': 1140.0, 'iscrowd': 0}


In [13]:
%%writefile /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r18vd_uav.yml

__include__: [
  '../dataset/coco_detection.yml',
  '../runtime.yml',
  './include/rtdetr_r18vd.yml',
]

# Override num_classes for single class (UAV)
num_classes: 1

# Training configuration
epoches: 72
use_amp: True
output_dir: '/content/drive/MyDrive/FYP_datasets/Vision-RGB/DUT/output'

# Optimizer
optimizer:
  type: AdamW
  params:
    -
      params: '^(?=.*backbone)(?=.*norm).*$'
      lr: 0.00001
      weight_decay: 0.
    -
      params: '^(?=.*backbone)(?!.*norm).*$'
      lr: 0.00001
    -
      params: '^(?!.*backbone).*$'
      lr: 0.0001
  lr: 0.0001
  betas: [0.9, 0.999]
  weight_decay: 0.0001

# Learning rate scheduler
lr_scheduler:
  type: MultiStepLR
  milestones: [60]
  gamma: 0.1

# LR warmup
lr_warmup_scheduler:
  type: LinearWarmup
  warmup_duration: 1000

# Training dataloader
train_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: '/content/DUT-train/train/img'
    ann_file: '/content/DUT-train/annotations/train.json'
    return_masks: False
    transforms:
      type: Compose
      ops:
        - {type: RandomPhotometricDistort, p: 0.5}
        - {type: RandomZoomOut, p: 0.5}
        - {type: RandomIoUCrop, p: 0.8}
        - {type: SanitizeBoundingBox}
        - {type: RandomHorizontalFlip}
        - {type: Resize, size: [640, 640]}
        - {type: ToTensor}
        - {type: Normalize, mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]}
  shuffle: True
  batch_size: 16
  num_workers: 4
  drop_last: True
  collate_fn:
    type: BatchImageCollateFuncion

# Validation dataloader
val_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: '/content/DUT-val/val/img'
    ann_file: '/content/DUT-val/annotations/val.json'
    return_masks: False
    transforms:
      type: Compose
      ops:
        - {type: Resize, size: [640, 640]}
        - {type: ToTensor}
        - {type: Normalize, mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]}
  shuffle: False
  batch_size: 16
  num_workers: 4
  drop_last: False
  collate_fn:
    type: BatchImageCollateFuncion

Writing /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r18vd_uav.yml


In [14]:
# Verify config file
!cat /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r18vd_uav.yml


__include__: [
  '../dataset/coco_detection.yml',
  '../runtime.yml',
  './include/rtdetr_r18vd.yml',
]

# Override num_classes for single class (UAV)
num_classes: 1

# Training configuration
epoches: 72
use_amp: True
output_dir: '/content/drive/MyDrive/FYP_datasets/Vision-RGB/DUT/output'

# Optimizer
optimizer:
  type: AdamW
  params:
    -
      params: '^(?=.*backbone)(?=.*norm).*$'
      lr: 0.00001
      weight_decay: 0.
    -
      params: '^(?=.*backbone)(?!.*norm).*$'
      lr: 0.00001
    -
      params: '^(?!.*backbone).*$'
      lr: 0.0001
  lr: 0.0001
  betas: [0.9, 0.999]
  weight_decay: 0.0001

# Learning rate scheduler
lr_scheduler:
  type: MultiStepLR
  milestones: [60]
  gamma: 0.1

# LR warmup
lr_warmup_scheduler:
  type: LinearWarmup
  warmup_duration: 1000

# Training dataloader
train_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: '/content/DUT-train/train/img'
    ann_file: '/content/DUT-train/annotations/train.json'
    return_m

In [13]:
# # Final verification
# import subprocess
# import sys

# test_script = """
# import sys
# sys.path.insert(0, '/content/RT-DETR/rtdetr_pytorch')

# try:
#     print("🔍 Final Import Test\\n")
#     print("="*70)

#     print("\\n1. Testing coco_dataset...")
#     from src.data.coco.coco_dataset import CocoDetection
#     print("   ✓ coco_dataset imported successfully")

#     print("\\n2. Testing transforms...")
#     from src.data.transforms import RandomPhotometricDistort, ToImage
#     print("   ✓ transforms imported successfully")

#     print("\\n3. Testing training module...")
#     import src.misc.dist as dist
#     print("   ✓ dist module imported successfully")

#     print("\\n" + "="*70)
#     print("🎉🎉🎉 ALL IMPORTS WORKING! 🎉🎉🎉")
#     print("RT-DETR is now ready to train!")
#     print("="*70)
#     exit(0)

# except Exception as e:
#     print(f"\\n❌ Still failing: {e}")
#     import traceback
#     traceback.print_exc()
#     exit(1)
# """

# result = subprocess.run([sys.executable, '-c', test_script], capture_output=True, text=True)
# print(result.stdout)
# if result.returncode != 0:
#     print(result.stderr)

🔍 Final Import Test


1. Testing coco_dataset...
   ✓ coco_dataset imported successfully

2. Testing transforms...

❌ Still failing: cannot import name 'ToImage' from 'src.data.transforms' (/content/RT-DETR/rtdetr_pytorch/src/data/transforms.py)

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.1
Traceback (most recent call last):
  File "<string>", line 14, in <module>
ImportError: cannot import name 'ToImage' from 'src.data.transforms' (/content/RT-DETR/rtdetr_pytorch/src/data/transforms.py)



In [15]:
# Start training
%cd /content/RT-DETR/rtdetr_pytorch

output_dir = '/content/drive/MyDrive/FYP_datasets/Vision-RGB/DUT/output'

!python tools/train.py \
    -c configs/rtdetr/rtdetr_r18vd_uav.yml \
    --amp \
    --seed 0

/content/RT-DETR/rtdetr_pytorch

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/content/RT-DETR/rtdetr_pytorch/tools/train.py", line 9, in <module>
    import src.misc.dist as dist
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/__init__.py", line 2, in <module>
    from . import data
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/__init__.py", line 2, in <module>
    from .coco import *
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/coco/__init__.py", line 1, in <module>
    from .coco_dataset import (
  File "/content/RT-DETR/rtdetr

In [16]:
os.chdir('/content/RT-DETR/rtdetr_pytorch')
output_dir = '/content/drive/MyDrive/FYP_datasets/Vision-RGB/DUT/output'
os.makedirs(output_dir, exist_ok=True)

print("=" * 70)
print("🚀 Starting RT-DETR Training")
print("=" * 70)

!python tools/train.py \
    -c configs/rtdetr/rtdetr_r18vd_uav.yml \
    --seed 0

print("\n✅ Training complete!")

🚀 Starting RT-DETR Training

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/content/RT-DETR/rtdetr_pytorch/tools/train.py", line 9, in <module>
    import src.misc.dist as dist
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/__init__.py", line 2, in <module>
    from . import data
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/__init__.py", line 2, in <module>
    from .coco import *
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/coco/__init__.py", line 1, in <module>
    from .coco_dataset import (
  File "/content/RT-DETR/rtdetr_pyt

In [17]:
!ls -la /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/include/

total 36
drwxr-xr-x 2 root root 4096 Dec  2 10:03 .
drwxr-xr-x 3 root root 4096 Dec  2 10:12 ..
-rw-r--r-- 1 root root 1110 Dec  2 10:03 dataloader_regnet.yml
-rw-r--r-- 1 root root 1110 Dec  2 10:03 dataloader.yml
-rw-r--r-- 1 root root  478 Dec  2 10:03 optimizer_regnet.yml
-rw-r--r-- 1 root root  528 Dec  2 10:03 optimizer.yml
-rw-r--r-- 1 root root 1257 Dec  2 10:03 rtdetr_dla34.yml
-rw-r--r-- 1 root root 1324 Dec  2 10:03 rtdetr_r50vd.yml
-rw-r--r-- 1 root root 1258 Dec  2 10:03 rtdetr_regnet.yml


In [18]:
# The output directory is set to your Drive folder

output_dir = '/content/drive/MyDrive/FYP_datasets/Vision-RGB/DUT/output'

!python tools/train.py \
    -c configs/rtdetr/rtdetr_r18vd_uav.yml \
    --output_dir {output_dir} \
    --amp


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/content/RT-DETR/rtdetr_pytorch/tools/train.py", line 9, in <module>
    import src.misc.dist as dist
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/__init__.py", line 2, in <module>
    from . import data
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/__init__.py", line 2, in <module>
    from .coco import *
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/coco/__init__.py", line 1, in <module>
    from .coco_dataset import (
  File "/content/RT-DETR/rtdetr_pytorch/tools/../src/data/coco/

In [19]:
import torchvision
print(f"Torchvision version: {torchvision.__version__}")
print(f"Has datapoints: {hasattr(torchvision, 'datapoints')}")
print(f"Has tv_tensors: {hasattr(torchvision, 'tv_tensors')}")

Torchvision version: 0.15.2+cu117
Has datapoints: False
Has tv_tensors: False



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [48]:
# Run this cell to see what's available in your torchvision
import torchvision

print("Available in torchvision:")
print([attr for attr in dir(torchvision) if not attr.startswith('_')])

print("\n\nChecking tv_tensors location:")
try:
    from torchvision import tv_tensors
    print(f"✓ tv_tensors found at: {tv_tensors}")
except ImportError as e:
    print(f"✗ tv_tensors not found: {e}")

print("\nChecking datapoints:")
try:
    from torchvision import datapoints
    print(f"✓ datapoints found")
except ImportError as e:
    print(f"✗ datapoints not found: {e}")

Available in torchvision:
['Module', 'datasets', 'disable_beta_transforms_warning', 'extension', 'get_image_backend', 'get_video_backend', 'io', 'models', 'ops', 'os', 'set_image_backend', 'set_video_backend', 'torch', 'transforms', 'utils', 'version', 'warnings']


Checking tv_tensors location:
✓ tv_tensors found at: <module 'torchvision.tv_tensors' from '/usr/local/lib/python3.12/dist-packages/torchvision/tv_tensors/__init__.py'>

Checking datapoints:
✗ datapoints not found: cannot import name 'datapoints' from 'torchvision' (/usr/local/lib/python3.12/dist-packages/torchvision/__init__.py)


In [49]:
# Quick diagnostic
try:
    from torchvision import tv_tensors
    print("✓ tv_tensors IS available!")
    print(f"  Location: {tv_tensors.__file__}")
    print(f"  Contents: {dir(tv_tensors)[:10]}")  # Show first 10 items
except ImportError as e:
    print(f"✗ tv_tensors NOT available: {e}")
    print("\nThis means we need a different approach!")

✓ tv_tensors IS available!
  Location: /usr/local/lib/python3.12/dist-packages/torchvision/tv_tensors/__init__.py
  Contents: ['BoundingBoxFormat', 'BoundingBoxes', 'Image', 'KeyPoints', 'Mask', 'TVTensor', 'Video', '__builtins__', '__cached__', '__doc__']
